In [16]:
import json

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import json

# Specify the path to your JSON file
file_path = '/content/drive/MyDrive/DS310_NLP/Project/VIMQA_Dataset/VIMQA_Dataset/vimqa_train.json'

# Open and read the JSON file
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Access or print the data
# print(data)


Processing contexts:  99%|█████████▉| 99/100 [20:01<00:12, 12.14s/it]


In [ ]:
print(len(data))

8041


In [ ]:
print(data[0])

{'_id': '01899ad3-1544-4a3f-80d0-78807558bf96', 'question': 'Film điện ảnh mà Margalit Ruth "Maggie" Gyllenhaal nhận được vai phụ độc lập vào năm 2001 có kinh phí là 4,5 triệu USD phải không?', 'answer': 'đúng', 'context': [['Donnie Darko', ['Ban đầu, bộ phim dự kiến phát hành đĩa phim luôn trước khi được phân phối bởi hãng Newmarket Films.', 'Kinh phí làm phim này là 4,5 triệu USD và được quay trong vòng 28 ngày.', 'Không như dự kiến, bộ phim chỉ thu về 4,1 triệu trên toàn cầu.', 'Mặc dù vậy, bộ phim nhận được rất nhiều lời khen ngợi từ giới phê bình và lượng người xem tăng rất nhanh.', "Do đó năm 2004, bộ phim phát hành bản Director's Cut."]], ['Kinh tế Việt Nam', ['Tổng sản phẩm nội địa GDP năm 2016 là 202 tỷ USD theo danh nghĩa hoặc 595 tỷ USD theo sức mua tương đương .']], ['Danh sách 100 phim hay nhất của Viện phim Mỹ', ['Danh sách 100 phim hay nhất của Viện phim Mỹ (tiếng Anh: AFI’s 100 Years...100 Movies) là danh sách 100 bộ phim Mỹ được coi là hay nhất của nền điện ảnh nước nà

In [ ]:
data[0]["context"]

[['Donnie Darko',
  ['Ban đầu, bộ phim dự kiến phát hành đĩa phim luôn trước khi được phân phối bởi hãng Newmarket Films.',
   'Kinh phí làm phim này là 4,5 triệu USD và được quay trong vòng 28 ngày.',
   'Không như dự kiến, bộ phim chỉ thu về 4,1 triệu trên toàn cầu.',
   'Mặc dù vậy, bộ phim nhận được rất nhiều lời khen ngợi từ giới phê bình và lượng người xem tăng rất nhanh.',
   "Do đó năm 2004, bộ phim phát hành bản Director's Cut."]],
 ['Kinh tế Việt Nam',
  ['Tổng sản phẩm nội địa GDP năm 2016 là 202 tỷ USD theo danh nghĩa hoặc 595 tỷ USD theo sức mua tương đương .']],
 ['Danh sách 100 phim hay nhất của Viện phim Mỹ',
  ['Danh sách 100 phim hay nhất của Viện phim Mỹ (tiếng Anh: AFI’s 100 Years...100 Movies) là danh sách 100 bộ phim Mỹ được coi là hay nhất của nền điện ảnh nước này trong vòng 100 năm qua do Viện phim Mỹ (American Film Institute, viết tắt là AFI) đưa ra năm 1998 nhân kỉ niệm 100 năm ra đời của điện ảnh.',
   'Danh sách này được lập ra sau quá trình bầu chọn từ dan

In [19]:
import os
os.environ["OPENAI_API_KEY"]=""

In [20]:
key = ""

In [21]:
context = [str(i["context"]) for i in data]

In [ ]:
len(context)

8041

In [ ]:
print(context[0])

[['Donnie Darko', ['Ban đầu, bộ phim dự kiến phát hành đĩa phim luôn trước khi được phân phối bởi hãng Newmarket Films.', 'Kinh phí làm phim này là 4,5 triệu USD và được quay trong vòng 28 ngày.', 'Không như dự kiến, bộ phim chỉ thu về 4,1 triệu trên toàn cầu.', 'Mặc dù vậy, bộ phim nhận được rất nhiều lời khen ngợi từ giới phê bình và lượng người xem tăng rất nhanh.', "Do đó năm 2004, bộ phim phát hành bản Director's Cut."]], ['Kinh tế Việt Nam', ['Tổng sản phẩm nội địa GDP năm 2016 là 202 tỷ USD theo danh nghĩa hoặc 595 tỷ USD theo sức mua tương đương .']], ['Danh sách 100 phim hay nhất của Viện phim Mỹ', ['Danh sách 100 phim hay nhất của Viện phim Mỹ (tiếng Anh: AFI’s 100 Years...100 Movies) là danh sách 100 bộ phim Mỹ được coi là hay nhất của nền điện ảnh nước này trong vòng 100 năm qua do Viện phim Mỹ (American Film Institute, viết tắt là AFI) đưa ra năm 1998 nhân kỉ niệm 100 năm ra đời của điện ảnh.', 'Danh sách này được lập ra sau quá trình bầu chọn từ danh sách đề cử gồm 400 bộ

In [22]:
from openai import AsyncOpenAI
client = AsyncOpenAI()

In [23]:
from collections.abc import KeysView
import asyncio
import openai

# Initialize the OpenAI client with your API key
openai.api_key = key

async def augment_context(context):
    completion = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {'role': 'system', 'content': """You will be given  context , return in json format with following keys:
             'question', 'answer', 'type' (only choose one of these: spatial_reasoning, causual_reasoning, temporal_reasoning, quantitative_reasoning), 'supporting_facts'. All of question, answer must be in Vietnamese
             just respond in json format, no explanation like this ```json .... ```
             one question only
             """
            },
            {"role": "user", "content": f"{context}"},
        ]
      )
    content = completion.choices[0].message.content
    return content

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from tqdm import tqdm
async def process_contexts(contexts, concurrency=5):
    sem = asyncio.Semaphore(concurrency)
    results = []

    # Create a progress bar
    pbar = tqdm(total=len(contexts), desc="Processing contexts")

    async def worker(ctx):
        async with sem:
            # Make the request
            response = await augment_context(ctx)
            results.append(response)
            # Update progress
            pbar.update(1)

    tasks = [asyncio.create_task(worker(c)) for c in contexts]
    await asyncio.gather(*tasks)
    pbar.close()
    return results

async def main():
    # # Replace with your actual list of contexts (8k, etc.)
    # contexts = context[100:200]
    # all_results = await process_contexts(contexts, concurrency=5)

    # # Write results to JSON
    # output_path = "/content/drive/MyDrive/DS310_NLP/Project/Output/output_3.json"
    # with open(output_path, "w", encoding="utf-8") as f:
    #     json.dump(all_results, f, ensure_ascii=False, indent=2)
    step = 50  # Số lượng context trong mỗi lần xử lý
    start = 3000
    end = 5000

    for i in range(start, end, step):
        contexts = context[i:i+step]
        all_results = await process_contexts(contexts, concurrency=5)

        # Đặt tên file động theo khoảng index
        output_path = f"/content/drive/MyDrive/DS310_NLP/Project/Output/output_{i}_{i+step}.json"
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(all_results, f, ensure_ascii=False, indent=2)
        print(f"Saved results for context[{i}:{i+step}] to {output_path}")

# In Colab, just call:
await main()